In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [2]:
# Load dataset
data = pd.read_excel('Dataset.xlsx', engine='openpyxl')
data.head()

,Team,Match Up,Game Date,W/L,MIN,PTS,FGM,FGA,FG%,3PM,...,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-
0,GSW,GSW vs. PHX,10/24/2023,L,240,104,36,101,35.6,10,...,78.6,18,31,49,19,11,6,11,23,-4
1,PHX,PHX @ GSW,10/24/2023,W,240,108,42,95,44.2,11,...,76.5,17,43,60,23,5,7,19,22,4
2,LAL,LAL @ DEN,10/24/2023,L,240,107,41,90,45.6,10,...,75,13,31,44,23,5,4,12,18,-12
3,DEN,DEN vs. LAL,10/24/2023,W,240,119,48,91,52.7,14,...,75,9,33,42,29,9,6,12,15,12
4,MEM,MEM vs. NOP,10/25/2023,L,240,104,38,91,41.8,12,...,80,8,29,37,23,8,7,13,19,-7


In [3]:
# Manipulate Data
# Convert game date to datetime
data['Game Date'] = pd.to_datetime(data['Game Date'], format='%m/%d/%Y')

# Extract home and guest teams
data['Home Team'] = data['Match Up'].apply(lambda x: x.split(' vs. ')[0] if 'vs.' in x else x.split(' @ ')[1])
data['Guest Team'] = data['Match Up'].apply(lambda x: x.split(' vs. ')[1] if 'vs.' in x else x.split(' @ ')[0])

# Add the correct Label column: 1 if home team won, 0 otherwise
data['Label'] = data.apply(
    lambda row: 1 if ((row['Team'] == row['Home Team']) and (row['W/L'] == 'W')) 
                    or ((row['Team'] == row['Guest Team']) and (row['W/L'] == 'L'))
                else 0,
    axis=1
)
data.head()

,Team,Match Up,Game Date,W/L,MIN,PTS,FGM,FGA,FG%,3PM,...,REB,AST,STL,BLK,TOV,PF,+/-,Home Team,Guest Team,Label
0,GSW,GSW vs. PHX,2023-10-24,L,240,104,36,101,35.6,10,...,49,19,11,6,11,23,-4,GSW,PHX,0
1,PHX,PHX @ GSW,2023-10-24,W,240,108,42,95,44.2,11,...,60,23,5,7,19,22,4,GSW,PHX,0
2,LAL,LAL @ DEN,2023-10-24,L,240,107,41,90,45.6,10,...,44,23,5,4,12,18,-12,DEN,LAL,1
3,DEN,DEN vs. LAL,2023-10-24,W,240,119,48,91,52.7,14,...,42,29,9,6,12,15,12,DEN,LAL,1
4,MEM,MEM vs. NOP,2023-10-25,L,240,104,38,91,41.8,12,...,37,23,8,7,13,19,-7,MEM,NOP,0


In [4]:
# Create a binary feature for home games
data['Is_Home'] = data['Match Up'].str.contains(' vs. ').astype(int)

# Prepare features and labels
X = data[['Is_Home']]  # Home/Away indicator
y = (data['W/L'] == 'W').astype(int)  # Convert W/L to binary outcome

# Fit a linear regression model
model = LinearRegression()
model.fit(X, y)

# The coefficient of 'Is_Home' represents the home advantage bias
bias = model.coef_[0]
print(f"Calculated Bias (Regression): {bias}")
new_bias = 1/21 * bias

# Remove the 'Is_Home' column
data = data.drop(columns=['Is_Home'])

Calculated Bias (Regression): 0.08617886178861783


In [5]:
# Baseline dataset
# Initialize the new dataset
new_dataset = []

# Iterate over each match-up
for _, row in data.iterrows():
    game_date = row['Game Date']
    home_team = row['Home Team']
    guest_team = row['Guest Team']
    
    # Filter games before the current game date for both teams
    home_team_games = data[(data['Team'] == home_team) & (data['Game Date'] < game_date)]
    guest_team_games = data[(data['Team'] == guest_team) & (data['Game Date'] < game_date)]
    
    # Calculate W/L rates (win rates) for both teams
    home_team_wins = (home_team_games['W/L'] == 'W').sum()
    home_team_total = len(home_team_games)
    guest_team_wins = (guest_team_games['W/L'] == 'W').sum()
    guest_team_total = len(guest_team_games)
    
    home_win_rate = home_team_wins / home_team_total if home_team_total > 0 else 0
    guest_win_rate = guest_team_wins / guest_team_total if guest_team_total > 0 else 0
    
    # Calculate W/L difference
    wl_difference = home_win_rate - guest_win_rate
    
    # Prepare the new row
    new_row = {
        'Game Date': game_date,
        'Home Team': home_team,
        'Guest Team': guest_team,
        'Label': int(row['Label']),  # Ensure Label is directly copied and kept as integer
        'W/L Difference': wl_difference
    }
    
    # Add statistics differences (like in the original approach)
    home_team_stats = home_team_games.iloc[:, 4:].mean(numeric_only=True)
    guest_team_stats = guest_team_games.iloc[:, 4:].mean(numeric_only=True)
    
    if not home_team_stats.empty and not guest_team_stats.empty:
        stats_diff = home_team_stats - guest_team_stats
        # Update new_row but ensure no conflict with 'Label'
        new_row.update({k: v for k, v in stats_diff.to_dict().items() if k != 'Label'})
    
    new_dataset.append(new_row)

# Convert to DataFrame
new_dataset_df = pd.DataFrame(new_dataset)

# Save to file for inspection
output_file = 'Dataset_without_New_Features.xlsx'
new_dataset_df.to_excel(output_file, index=False)

output_file

'Dataset_without_New_Features.xlsx'

In [14]:
# dataset with new feature: stability score
# Initialize the new dataset
new_dataset = []

# Iterate over each match-up
for _, row in data.iterrows():
    game_date = row['Game Date']
    home_team = row['Home Team']
    guest_team = row['Guest Team']
    
    # Filter games before the current game date for both teams
    home_team_games = data[(data['Team'] == home_team) & (data['Game Date'] < game_date)]
    guest_team_games = data[(data['Team'] == guest_team) & (data['Game Date'] < game_date)]
    
    # Calculate W/L rates (win rates) for both teams
    home_team_wins = (home_team_games['W/L'] == 'W').sum()
    home_team_total = len(home_team_games)
    guest_team_wins = (guest_team_games['W/L'] == 'W').sum()
    guest_team_total = len(guest_team_games)
    
    home_win_rate = home_team_wins / home_team_total if home_team_total > 0 else 0
    guest_win_rate = guest_team_wins / guest_team_total if guest_team_total > 0 else 0
    
    # Calculate W/L difference
    wl_difference = home_win_rate - guest_win_rate
    
    # Calculate Stability using mean and variance
    if not home_team_games.empty:
        home_mean_performance = home_team_games.iloc[:, 4:].mean(numeric_only=True)
        home_variance_performance = home_team_games.iloc[:, 4:].var(numeric_only=True)
        home_stability = (home_mean_performance / (home_variance_performance + 1e-6)).mean()
        home_stability = min(max(home_stability, -100), 100)
    else:
        home_stability = 0

    if not guest_team_games.empty:
        guest_mean_performance = guest_team_games.iloc[:, 4:].mean(numeric_only=True)
        guest_variance_performance = guest_team_games.iloc[:, 4:].var(numeric_only=True)
        guest_stability = (guest_mean_performance / (guest_variance_performance + 1e-6)).mean()
        guest_stability = min(max(guest_stability, -100), 100)  # Limit stability within [-100, 100]
    else:
        guest_stability = 0

    # Stability difference
    stability = home_stability - guest_stability
    
    # Prepare the new row
    new_row = {
        'Game Date': game_date,
        'Home Team': home_team,
        'Guest Team': guest_team,
        'Label': int(row['Label']),  # Ensure Label is directly copied and kept as integer
        'W/L Difference': wl_difference,
        'Stability': stability
    }
    
    # Add statistics differences (like in the original approach)
    home_team_stats = home_team_games.iloc[:, 4:].mean(numeric_only=True)
    guest_team_stats = guest_team_games.iloc[:, 4:].mean(numeric_only=True)
    
    if not home_team_stats.empty and not guest_team_stats.empty:
        stats_diff = home_team_stats - guest_team_stats
        # Update new_row but ensure no conflict with 'Label'
        new_row.update({k: v for k, v in stats_diff.to_dict().items() if k != 'Label'})
    
    new_dataset.append(new_row)

# Convert to DataFrame
new_dataset_df = pd.DataFrame(new_dataset)

# Save to file for inspection
output_file = 'Dataset_With_Stability.xlsx'
new_dataset_df.to_excel(output_file, index=False)

output_file

'Dataset_With_Stability.xlsx'

In [ ]:
# Initialize the new dataset
new_dataset = []

# Iterate over each match-up
for _, row in data.iterrows():
    game_date = row['Game Date']
    home_team = row['Home Team']
    guest_team = row['Guest Team']
    
    # Filter games before the current game date for both teams
    home_team_games = data[(data['Team'] == home_team) & (data['Game Date'] < game_date)]
    guest_team_games = data[(data['Team'] == guest_team) & (data['Game Date'] < game_date)]
    
    # Calculate W/L rates (win rates) for both teams
    home_team_wins = (home_team_games['W/L'] == 'W').sum()
    home_team_total = len(home_team_games)
    guest_team_wins = (guest_team_games['W/L'] == 'W').sum()
    guest_team_total = len(guest_team_games)
    
    home_win_rate = home_team_wins / home_team_total if home_team_total > 0 else 0
    guest_win_rate = guest_team_wins / guest_team_total if guest_team_total > 0 else 0
    
    # Calculate W/L difference
    wl_difference = home_win_rate - guest_win_rate
    
    # Calculate Stability: (variance difference between home and guest teams)
    home_stability = home_team_games.iloc[:, 4:].var(numeric_only=True).mean() if not home_team_games.empty else 0
    guest_stability = guest_team_games.iloc[:, 4:].var(numeric_only=True).mean() if not guest_team_games.empty else 0
    stability = home_stability + bias - guest_stability  # Stability difference
    
    # Prepare the new row
    new_row = {
        'Game Date': game_date,
        'Home Team': home_team,
        'Guest Team': guest_team,
        'Label': int(row['Label']),  # Ensure Label is directly copied and kept as integer
        'W/L Difference': wl_difference,
        'Stability': stability
    }
    
    # Add statistics differences (like in the original approach)
    home_team_stats = home_team_games.iloc[:, 4:].mean(numeric_only=True)
    guest_team_stats = guest_team_games.iloc[:, 4:].mean(numeric_only=True)
    
    if not home_team_stats.empty and not guest_team_stats.empty:
        stats_diff = home_team_stats - guest_team_stats
        # Update new_row but ensure no conflict with 'Label'
        new_row.update({k: v for k, v in stats_diff.to_dict().items() if k != 'Label'})
    
    new_dataset.append(new_row)

# Convert to DataFrame
new_dataset_df = pd.DataFrame(new_dataset)

# Save to file for inspection
output_file = 'Dataset_With_Stability_and_HomeAdvantage.xlsx'
new_dataset_df.to_excel(output_file, index=False)

output_file

'Dataset_with_Home_Advantage.xlsx'

In [21]:
# Initialize the new dataset
new_dataset = []

# Iterate over each match-up
for _, row in data.iterrows():
    game_date = row['Game Date']
    home_team = row['Home Team']
    guest_team = row['Guest Team']
    
    # Filter games before the current game date for both teams
    home_team_games = data[(data['Team'] == home_team) & (data['Game Date'] < game_date)]
    guest_team_games = data[(data['Team'] == guest_team) & (data['Game Date'] < game_date)]
    
    # Calculate W/L rates (win rates) for both teams
    home_team_wins = (home_team_games['W/L'] == 'W').sum()
    home_team_total = len(home_team_games)
    guest_team_wins = (guest_team_games['W/L'] == 'W').sum()
    guest_team_total = len(guest_team_games)
    
    home_win_rate = home_team_wins / home_team_total if home_team_total > 0 else 0
    guest_win_rate = guest_team_wins / guest_team_total if guest_team_total > 0 else 0
    
    # Calculate W/L difference
    wl_difference = home_win_rate - guest_win_rate

    # Determine Home Advantage: 1 for home team, 0 for guest team or neutral field
    home_advantage = 1 if 'vs.' in row['Match Up'] else 0
    
    # Prepare the new row
    new_row = {
        'Game Date': game_date,
        'Home Team': home_team,
        'Guest Team': guest_team,
        'Label': int(row['Label']),  # Ensure Label is directly copied and kept as integer
        'W/L Difference': wl_difference,
        'Home Advantage': home_advantage  # Add the new Home Advantage feature
    }
    
    # Add statistics differences (like in the original approach)
    if not home_team_games.empty and not guest_team_games.empty:
        # Ensure numeric columns only
        home_team_stats = home_team_games.iloc[:, 4:].mean(numeric_only=True)
        guest_team_stats = guest_team_games.iloc[:, 4:].mean(numeric_only=True)
        
        if not home_team_stats.empty and not guest_team_stats.empty:
            stats_diff = home_team_stats - guest_team_stats
            # Update new_row but ensure no conflict with 'Label'
            new_row.update({k: v for k, v in stats_diff.to_dict().items() if k != 'Label'})
    
    new_dataset.append(new_row)

# Convert to DataFrame
new_dataset_df = pd.DataFrame(new_dataset)

# Save to file for inspection
output_file = 'Dataset_with_Home_Advantage_and_Stats.xlsx'
new_dataset_df.to_excel(output_file, index=False)

output_file

'Dataset_with_Home_Advantage_and_Stats.xlsx'

In [6]:
# Remove rows with NaN values and drop duplicate rows
dataset_df = new_dataset_df.dropna().drop_duplicates()

# Save to file for inspection
output_file = 'cleaned_final_dataset.xlsx'
dataset_df.to_excel(output_file, index=False)
output_file

'cleaned_final_dataset.xlsx'

In [7]:
# Initialize a list to store accuracies
accuracies = []

X = dataset_df.drop(columns=['Label', 'Game Date', 'Home Team', 'Guest Team'])  # Features: all columns except 'Label'
y = dataset_df['Label']  # Labels: the 'Label' column
# Loop to train and test the model 20 times
for i in range(20):
    # Split the dataset into training and testing sets with a different random state each time
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=i)
    
    # Fit a Random Forest Classifier
    rf_classifier = RandomForestClassifier(random_state=42)
    rf_classifier.fit(X_train, y_train)
    
    # Predict on the test set
    y_pred = rf_classifier.predict(X_test)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)

# Calculate the average accuracy over 20 iterations
average_accuracy = np.mean(accuracies)

# Print the average accuracy
print(f"Average Accuracy of the Random Forest Classifier over 20 runs: {average_accuracy:.2f}")

Average Accuracy of the Random Forest Classifier over 20 runs: 0.62
